Our primary dataset:

Eurosat

[DOI](10.1109/IGARSS.2018.8519248)

[dataset](https://zenodo.org/records/7711810#.ZAm3k-zMKEA)


download EuroSAT_MS.zip (the full spectral dataset) and extract it into this directory

In [1]:
import torch
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
# from tqdm import tqdm
 
# !pip install torchvision
import torchvision
 
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
 
# !pip install torchmetrics
import torchmetrics
import os

x = torch.rand(5, 3)
print(x)



tensor([[0.4012, 0.7886, 0.1489],
        [0.3766, 0.2782, 0.7422],
        [0.5040, 0.3768, 0.0892],
        [0.9001, 0.0771, 0.3138],
        [0.8700, 0.9592, 0.1054]])


In [10]:
# import zipfile
 
# Define the path to your zip file
# zip_file_path = 'EuroSAT_RGB.zip'
 
 
# extract_to_directory = 'extracted_files'
 
# try:
#     with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#         zip_ref.extractall(extract_to_directory)
#     print(f"Successfully extracted all contents of '{zip_file_path}' to '{extract_to_directory}'")
# except zipfile.BadZipFile:
#     print(f"Error: '{zip_file_path}' is not a valid zip file.")
# except FileNotFoundError:
#     print(f"Error: Zip file '{zip_file_path}' not found.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")
 
from sklearn.model_selection import train_test_split
 
# Gather all image file paths
image_dir = './EuroSAT_RGB/'
all_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir)]
all_paths = all_paths[1:]
print(all_paths)
all_files = []
labels = []

for path in all_paths:
  all_files.extend([os.path.join(path, f) for f in os.listdir(path)])
  index = path.find('EuroSAT_RGB/')
  if index != -1:
      start_index = index + len('EuroSAT_RGB/')
      result = path[start_index:]
  else:
      result = "" # Target string not found
  labels.extend([result] * len(os.listdir(path)))
 

# Get labels (assuming class names are part of the file paths)
labels = [os.path.basename(os.path.dirname(f)) for f in all_files]
print(len(labels)) 

# Split the file paths, using stratify to maintain class distribution
X_train, X_test, y_train, y_test = train_test_split(
    all_files, labels, test_size=0.2, random_state=42, stratify=labels
) 
print("X:",X_train[0]) 
print(len(X_train)) 
print(len(X_test)) 
print("y:",y_train[0]) 
print(len(y_train)) 
print(len(y_test)) 


['./EuroSAT_RGB/Forest', './EuroSAT_RGB/Highway', './EuroSAT_RGB/Residential', './EuroSAT_RGB/Industrial', './EuroSAT_RGB/PermanentCrop', './EuroSAT_RGB/SeaLake', './EuroSAT_RGB/Pasture', './EuroSAT_RGB/AnnualCrop', './EuroSAT_RGB/HerbaceousVegetation', './EuroSAT_RGB/River']
27000
X: ./EuroSAT_RGB/AnnualCrop/AnnualCrop_845.jpg
21600
5400
y: AnnualCrop
21600
5400


In [ ]:
# from PIL import Image
# from torchvision.transforms import ToTensor

# image = Image.open('EuroSAT_RGB/AnnualCrop/AnnualCrop_1.jpg')
# image = ToTensor()(image)
# print(image.shape)

torch.Size([3, 64, 64])


In [4]:
from torch import nn
class CNN(nn.Module):
   def __init__(self, in_channels, num_classes):
 
       """
       Building blocks of convolutional neural network.
 
       Parameters:
           * in_channels: Number of channels in the input image (for grayscale images, 1)
           * num_classes: Number of classes to predict. In our problem, 10 (i.e digits from  0 to 9).
       """
       super(CNN, self).__init__()
 
       # 1st convolutional layer
       self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, padding=1)
       # Max pooling layer
       self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
       # 2nd convolutional layer
       self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
       # Fully connected layer
       self.fc1 = nn.Linear(16 * 7 * 7, num_classes)
 
   def forward(self, x):
       """
       Define the forward pass of the neural network.
 
       Parameters:
           x: Input tensor.
 
       Returns:
           torch.Tensor
               The output tensor after passing through the network.
       """
       x = F.relu(self.conv1(x))  # Apply first convolution and ReLU activation
       x = self.pool(x)           # Apply max pooling
       x = F.relu(self.conv2(x))  # Apply second convolution and ReLU activation
       x = self.pool(x)           # Apply max pooling
       x = x.reshape(x.shape[0], -1)  # Flatten the tensor
       x = self.fc1(x)            # Apply fully connected layer
       return x



In [ ]:
# batch_size = 60

# train_dataset = datasets.MNIST(root="EuroSAT_RGB/", download=True, train=True, transform=transforms.ToTensor())

# train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# test_dataset = datasets.MNIST(root="EuroSAT_RGB/", download=True, train=False, transform=transforms.ToTensor())

# test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNN(in_channels=3, num_classes=10).to(device)
print(model)
# >>> CNN(
#   (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#   (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
#   (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#   (fc1): Linear(in_features=784, out_features=10, bias=True)
# )

CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)


In [ ]:

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

['MNIST', 'Forest', 'Highway', 'Residential', 'Industrial', 'PermanentCrop', 'SeaLake', 'Pasture', 'AnnualCrop', 'HerbaceousVegetation', 'River']


In [ ]:
# import tqdm

from torchvision.transforms import ToTensor
num_epochs=10
for epoch in range(num_epochs):
 # Iterate over training batches
   print(f"Epoch [{epoch + 1}/{num_epochs}]")

   for image,label in zip(X_train,y_train):
      image = Image.open(image)
      image = ToTensor()(image)
      # data = image.to(device)
      # targets = label.to(device)
      scores = model(image)
      loss = criterion(scores, y_train[i])
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()



Epoch [1/10]


[W925 17:23:40.056207400 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W925 17:23:40.074076570 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x256 and 784x10)